In [1]:
%pip install yfinance
import sqlite3
import yfinance as yf
from datetime import datetime, timedelta
date_range=0
update_all= True # True is update all, 
# Function to parse date with multiple formats
def parse_date(date_str):
    """
    Parse a date string with multiple possible formats.
    """
    for fmt in ('%Y-%m-%d', '%m/%d/%Y', '%d/%m/%Y', '%Y-%m-%d %H:%M:%S'):
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    return None

# Function to fetch Yahoo Finance data
def fetch_yahoo_finance_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        cashflow = stock.quarterly_cashflow
        balance_sheet = stock.balance_sheet
        shares_outstanding = stock.info.get('sharesOutstanding')
        current_stock_price = stock.info.get('currentPrice')  # Current stock price

        # Calculate TTM Free Cash Flow
        if "Free Cash Flow" in cashflow.index:
            ttm_free_cash_flow = cashflow.loc["Free Cash Flow"].iloc[:4].sum()
        else:
            ttm_free_cash_flow = None

        # Get the latest balance sheet data
        latest_date_balance = balance_sheet.columns[0]
        latest_balance_sheet = balance_sheet[latest_date_balance]
        cash_equivalents = latest_balance_sheet.get("Cash Cash Equivalents And Short Term Investments", 0)
        total_debt = latest_balance_sheet.get("Total Debt", 0)
        latest_cashflow_date = cashflow.columns[0].date()   
        print (latest_cashflow_date)
        return {
            "Symbol": symbol,
            "Free Cash Flow": ttm_free_cash_flow or 0,
            "Cash Equivalents": float(cash_equivalents),
            "Total Debt": float(total_debt),
            "Shares Outstanding": shares_outstanding or 0,
            "Update Date": datetime.now().strftime('%Y-%m-%d'),  # Simplified human-readable date
            "Report_date": latest_cashflow_date,
        }
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Function to write or update data in the DCF_Data table
def write_or_update_database(db_path, data, er_date, symbol):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 检查 Manullly 列是否为空
        cursor.execute("SELECT Manullly FROM DCF_Data WHERE Symbol = ?", (data["Symbol"],))
        manullly_value = cursor.fetchone()

        if manullly_value and manullly_value[0]:  # 如果 Manullly 列不为空，跳过更新
            print(f"Skipping update for {symbol} as Manullly is not empty.")
            return

        # 如果没有跳过，继续检查是否需要更新
        cursor.execute("SELECT Update_date FROM DCF_Data WHERE Symbol = ?", (data["Symbol"],))
        existing_entry = cursor.fetchone()

        # 解析 ER_Date 为 datetime 对象
        er_date_parsed = parse_date(er_date) if er_date else None

        update_needed = True
        if existing_entry:
            # 尝试解析现有的 Update_date
            existing_update_date = parse_date(existing_entry[0])
            if existing_update_date and er_date_parsed:
                # 检查是否需要更新
                if existing_update_date >= er_date_parsed - timedelta(days=date_range):
                    print(symbol, "Update in time, no action")
                    update_needed = False
                    update_needed = update_all  # 更新所有

        if update_needed:
            # 更新或插入记录
            if existing_entry:
                query = """
                UPDATE DCF_Data
                SET FCF = ?, Cash_Equivalents = ?, Total_debt = ?, Shares_outstanding = ?, Update_date = ?, Report_date = ?
                WHERE Symbol = ?
                """
                cursor.execute(query, (
                    data["Free Cash Flow"],
                    data["Cash Equivalents"],
                    data["Total Debt"],
                    data["Shares Outstanding"],
                    data["Update Date"],
                    data["Report_date"],
                    data["Symbol"],
                ))
                print(f"Data for {data['Symbol']} updated successfully.")
            else:
                query = """
                INSERT INTO DCF_Data (Symbol, FCF, Cash_Equivalents, Total_debt, Shares_outstanding, Update_date, Report_date)
                VALUES (?, ?, ?, ?, ?, ?, ?)
                """
                cursor.execute(query, (
                    data["Symbol"],
                    data["Free Cash Flow"],
                    data["Cash Equivalents"],
                    data["Total Debt"],
                    data["Shares Outstanding"],
                    data["Update Date"],
                    data["Report_date"],
                ))
                print(f"Data for {data['Symbol']} inserted successfully.")

        conn.commit()
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    finally:
        conn.close()


# Function to fetch symbols and ER_Date from CValue table
def fetch_symbols_and_er_date(db_path):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        query = """
        SELECT Symbol, ER_Date
        FROM CValue
        WHERE WACC IS NOT NULL AND Growth5Y IS NOT NULL;
        """
        cursor.execute(query)
        return cursor.fetchall()  # List of tuples (Symbol, ER_Date)
    except sqlite3.Error as e:
        print(f"SQLite error while fetching symbols and ER_Date: {e}")
        return []
    finally:
        conn.close()

# Main function
def main():
    db_path = "plan.db"  # Path to your SQLite database
    symbol_er_date_list = fetch_symbols_and_er_date(db_path)

    for symbol, er_date in symbol_er_date_list:
        data = fetch_yahoo_finance_data(symbol)
        if data:
            write_or_update_database(db_path, data, er_date,symbol)

if __name__ == "__main__":
    main()



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Rhino\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: yfinance in c:\users\rhino\appdata\local\programs\python\python312\lib\site-packages (0.2.49)



429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MSFT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MSFT&crumb=Edge%3A+Too+Many+Requests


Error fetching data for MSFT: Expecting value: line 1 column 1 (char 0)


429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AAPL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AAPL&crumb=Edge%3A+Too+Many+Requests


Error fetching data for AAPL: Expecting value: line 1 column 1 (char 0)


429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PYPL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PYPL&crumb=Edge%3A+Too+Many+Requests


Error fetching data for PYPL: Expecting value: line 1 column 1 (char 0)


KeyboardInterrupt: 

In [ ]:
from datetime import date, datetime, timedelta
import sqlite3
import pandas as pd
import yfinance as yf
#成熟企业：WACC 通常在 8%-12%。
#高风险行业（如科技、创业公司）：WACC 较高（12%-20%）。
#稳定行业（如公用事业、消费品）：WACC 较低（6%-8%）。
#Growth Rate 对于轻资产行业可以用EPS 3-5 年增速代替。重资产行业用Free Cash Flow Per Share Growth Rate (FWD) 的五年平均值代替
# 最优解是 Growth 里的EPS FWD Long Term Growth (3-5Y CAGR) 其次是Free Cash Flow Per Share Growth Rate (FWD) 最次是Revenu5Y 但是要求很高的WACC
# WACC或者折现率也可以理解为你对于回报的期望值比如说不能低于十年期国债的收益率多2%作为风险回报,  或者不能低于同行业的平均值
# Constants

GDP = 0.03  # Permanent growth rate (long-term GDP growth)
current_date = date.today().strftime('%Y-%m-%d')

# Set the threshold for DCF update in days (0 = update all)
days_threshold = 0  # Example: Update if DCFDate is older than 10 days

# Database path
db_path = "plan.db"

# Query to fetch data
query = """
SELECT Symbol, WACC, Growth5Y,Growth10Y,DCFDate
FROM CValue
WHERE WACC IS NOT NULL AND Growth5Y IS NOT NULL;
"""

# Function to fetch Yahoo Finance data
import sqlite3

# Function to fetch data from the database
def fetch_data_from_database(db_path, symbol):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Query to fetch data for the given symbol
        query = """
        SELECT FCF, Cash_Equivalents, Total_debt, Shares_outstanding, Report_date, Manullly
        FROM DCF_Data
        WHERE Symbol = ?;
        """
        cursor.execute(query, (symbol,))
        result = cursor.fetchone()

        if result:
            # Map the result to a dictionary
            return {
                "Free Cash Flow": result[0],
                "Cash Equivalents": result[1],
                "Total Debt": result[2],
                "Shares Outstanding": result[3],
                "Report_date": result[4],
                "Manually": result[5],
            }
        else:
            print(f"No data found for symbol: {symbol}")
            return None
    except sqlite3.Error as e:
        print(f"SQLite error while fetching data for {symbol}: {e}")
        return None
    finally:
        conn.close()


# Main program
try:
    with sqlite3.connect(db_path) as conn:
        # Fetch data from the database
        df_wacc_growth = pd.read_sql_query(query, conn)

        # Initialize results list
        results = []

        # Iterate over each stock in the database
        for i, row in df_wacc_growth.iterrows():
            symbol = row['Symbol']
            print(f"\nProcessing {symbol}...")  # Log current Symbol
            wacc = row['WACC'] / 100  # Convert to decimal
            growth_rate = row['Growth5Y'] / 100  # Convert to decimal
            growth_rate_10 = row['Growth10Y'] / 100 if isinstance(row['Growth10Y'], (int, float)) else growth_rate * 0.8 # 没有二段数据就用80%
            dcf_date = row.get('DCFDate')  # Read DCFDate

            # Skip if Growth Rate is negative
            if growth_rate < 0:
                print(f"Skipping {symbol}: Growth Rate is negative.")
                continue

            # Check if DCF needs to be updated
            if dcf_date:
                try:
                    dcf_date = datetime.strptime(dcf_date, '%Y-%m-%d')
                    days_since_update = (datetime.now() - dcf_date).days
                    if days_threshold > 0 and days_since_update <= days_threshold:
                        print(f"Skipping {symbol}: Last updated {days_since_update} days ago.")
                        continue
                except ValueError:
                    print(f"Invalid DCFDate for {symbol}, recalculating.")
            else:
                print(f"Missing DCFDate for {symbol}, recalculating.")

            # Fetch Yahoo Finance data and process normally
            finance_data = fetch_data_from_database(db_path, symbol)
            
            if not finance_data:
                print(f"Error fetching data for {symbol}, skipping.")
                continue

            free_cash_flow = finance_data["Free Cash Flow"]
            cash_equivalents = finance_data["Cash Equivalents"]
            total_debt = finance_data["Total Debt"]
            shares_outstanding = finance_data["Shares Outstanding"]
            report_date=finance_data["Report_date"]

            # Skip if any critical data is missing
            if None in (free_cash_flow, cash_equivalents, total_debt, shares_outstanding):
                print(f"Skipping {symbol}: Missing financial data.")
                continue

            # Print intermediate data for debugging
            
            print(f"  TTM Free Cash Flow: {free_cash_flow}")
            print(f"  Cash Equivalents: {cash_equivalents}")
            print(f"  Total Debt: {total_debt}")
            print(f"  Shares Outstanding: {shares_outstanding}")
            

            # Calculate cash flows for the next 10 years
            future_cash_flows = []
            current_cash_flow = free_cash_flow
            for year in range(1, 6):  # First 5 years
                current_cash_flow *= (1 + growth_rate)
                future_cash_flows.append(current_cash_flow)
            for year in range(6, 11):  # Next 5 years with reduced growth
                current_cash_flow *= (1 + growth_rate_10)
                future_cash_flows.append(current_cash_flow)

            # Print projected future cash flows for debugging
            print(f"  Projected Future Cash Flows (Years 1-10): {future_cash_flows}")

            # Calculate terminal value
            terminal_value = future_cash_flows[-1] * (1 + GDP) / (wacc - GDP)
            print(f"  Terminal Value (TV): {terminal_value}")  # Log terminal value
            future_cash_flows[-1] += terminal_value

            # Calculate discounted cash flows
            discounted_cash_flows = [
                cash_flow / (1 + wacc) ** year for year, cash_flow in enumerate(future_cash_flows, start=1)
            ]
            enterprise_value = sum(discounted_cash_flows)

            # Print enterprise value for debugging
            print(f"  Enterprise Value (EV): {enterprise_value}")

            # Calculate equity value and DCF price
            equity_value = enterprise_value + cash_equivalents - total_debt
            try:
                dcf_price = equity_value / shares_outstanding
            except:
                dcf_price = 0
            # Append result
            results.append({
                "Symbol": symbol,
                "WACC": wacc * 100,
                "Growth Rate (%)": growth_rate * 100,
                "DCF Price": dcf_price,    
                "report_date":report_date,
            })

            # Log next Symbol (if exists)
            if i + 1 < len(df_wacc_growth):
                next_symbol = df_wacc_growth.iloc[i + 1]['Symbol']
                print(f"Next symbol to process: {next_symbol}")
            else:
                print("No more symbols to process.")

        # Write DCF results back to the database
        cursor = conn.cursor()
        cursor.execute("PRAGMA table_info(CValue);")
        columns = [col[1] for col in cursor.fetchall()]

        if "DCF" not in columns:
            cursor.execute("ALTER TABLE CValue ADD COLUMN DCF REAL;")
        if "DCFDate" not in columns:
            cursor.execute("ALTER TABLE CValue ADD COLUMN DCFDate TEXT;")

        for result in results:
            cursor.execute(
                "UPDATE CValue SET DCF = ?, DCFDate = ? WHERE Symbol = ?;",
                (result["DCF Price"], result["report_date"], result["Symbol"])
            )
            print (result)
        #conn.commit()

        print("DCF values successfully written back to the database.")

except Exception as e:
    print("Error:", str(e))


